In [ ]:
import pandas as pd
from windrose import WindroseAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np
import seaborn as sns

In [ ]:
# Import weather data from .txt file, skip first two rows
df = pd.read_csv("23_10_data.txt", header = None, skiprows = 2, parse_dates = [[0, 1]], dayfirst = True, sep = "\t")
df.head()

In [ ]:
# Replace headers
header = ['DateTime', 'Temp Out', 'Hi Temp', 'Low Temp', 'Out Hum', 'Dew Pt.', 'Wind Speed',
 'Wind Dir', 'Wind Run', 'Hi Speed', 'Hi Dir', 'Wind Chill', 'Heat Index', 'THW Index',
 'Bar', 'Rain', 'Rain Rate', 'Heat D-D', 'Cool D-D', 'In Temp', 'In Hum', 'In Dew',
 'In Heat', 'In EMC', 'In Air Density', 'Wind Samp', 'Wind Tx', 'ISS Recept', 'Arc. Int.']

df.columns = header
df.head()

# Other values
# Solar Rad.', 'Solar Energy', 'Hi Solar Rad.', 'UV Index', 'UV Dose', 'Hi UV', 

In [ ]:
# Get Month and Year of data for Labels and filenames
FileMonthYear = df['DateTime'][0].strftime('%y_%m')
MonthYear = str(df['DateTime'][0].month_name() + ' ' + str(df['DateTime'][0].year))

In [ ]:
# Replace blank wind direction values with NaN
df['Wind Dir'].replace('---', np.nan, inplace=True)

# Replace Compass points with bearings
df['Wind Dir'].replace(['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE', 'S', 
                            'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW'], 
                        [0, 22.5, 45, 67.5, 90, 112.5, 135, 157.5, 180,
                             202.5, 225, 247.5, 270, 292.5, 315, 337.5], inplace=True) 

In [ ]:
# Plot graph
from matplotlib.ticker import FormatStrFormatter

ax = WindroseAxes.from_ax()
ax.bar(df['Wind Dir'], df['Wind Speed'], cmap = cm.Greys, normed=True, 
           bins=np.arange(0, max(df['Wind Speed']), 5), opening = 0.8, edgecolor = 'black')

ax.set_xticklabels(['E', 'NE', 'N', 'NW','W', 'SW', 'S', 'SE'], fontsize = 16)

ax.set_legend()
ax.set_legend(title = 'Wind Speed in mph', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

ax.yaxis.set_major_formatter(FormatStrFormatter('%d %'))

# Notes to help fit axis to data:
# (x, y, step = z)
# x is first tick, y is the max (add on one step size more), z is the step between ticks

ax.set_yticks(np.arange(5, 20, step = 5))
ax.set_yticklabels(np.arange(5, 20, step = 5), fontsize = 16, backgroundcolor = 'white')

ax.annotate(MonthYear, xy = (0,1), xycoords='axes fraction', fontsize = 16)

plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%d %%'))

ax.figure.savefig(FileMonthYear + '_windrose.png', bbox_inches='tight', dpi = 600)